In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
import os
import re
import scipy

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Extract the digits from the filename
def extract_digits(string):
    return ''.join(re.findall(r'\d+', string))

In [ ]:
# Apply hair segmentation mask to original image
def load_and_preprocess(img_path, mask_path):

  # Read image from path
  img = cv2.imread(img_path)

  # Read hair segmentation mask from path
  mask = cv2.imread(mask_path)

  # Resize image to match the size of the mask
  img = cv2.resize(img, (mask.shape[1], mask.shape[0]))

  # Convert color space from BGR to HSV
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  #img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) #UNCOMMENT THIS LINE TO GENERATE masks on the RGB color space!

  # Filter out the hair pixels from the rest of the image
  hair_mask = cv2.bitwise_and(img, mask)

  return hair_mask

In [ ]:
def generate_mask(hair_mask):

  # Store dimensions of input image
  height, width, _ = hair_mask.shape

  # Generate the x and y coordinates of every pixel in the input image
  x_coord, y_coord = np.meshgrid(np.arange(width), np.arange(height))

  # Stack the x and y coordinates on top of each to create a N x M x 2 tensor
  coords = np.stack((x_coord, y_coord), axis=2)

  # Concatenate the x and y coordinates to the 'channel' dimension after the color channels (ex. N x M x 3 --> N x M x 5)
  hair_mask = np.concatenate((hair_mask, coords), axis=2)
  #print(f"The dimensions of the image are {img.shape}")

  # Extract the hair pixels
  hair_pixels = hair_mask[~np.all(hair_mask[:,:,0:3] == 0, axis=2)]
  #print(hair_pixels.shape)

  # Fit GMM with 2 clusters using the 'Hue' and 'Saturation' dimensions of the HSV color model
  kmeans = KMeans(n_clusters=2).fit(hair_pixels[:,[0,1,2]])                                          # UNCOMMENT THIS LINE to use the KMEANS model (+ the related 'cluster_pixels' line below)
  #gm = GaussianMixture(n_components=2).fit_predict(hair_pixels[:,[0,1,2]])                            # NOTICE: [0,1] to [0,1,2] if using the RGB color space! (In the HSV color space, only the first two channels are currently being used)

  # Add column indicating cluster assignments to "hair_pixels" matrix
  cluster_pixels = np.concatenate((hair_pixels, kmeans.labels_[:, np.newaxis]), axis=1)            # UNCOMMENT THIS LINE to use the KMEANS model (+ the related 'KMeans' line above)
  #cluster_pixels = np.concatenate((hair_pixels, gm[:, np.newaxis]), axis=1)

  # Extract the pixels belonging to each cluster
  cluster_0 = cluster_pixels[cluster_pixels[:,5] == 0]
  cluster_1 = cluster_pixels[cluster_pixels[:,5] == 1]

  # Assign the pixels in the smaller cluster to a new variable
  min_cluster = cluster_0[:,0:5] if cluster_0.shape[0] < cluster_1.shape[0] else cluster_1[:,0:5]

  # Assign the pixels in the larger cluster to a new variable
  maj_cluster = cluster_0[:,0:5] if cluster_0.shape[0] > cluster_1.shape[0] else cluster_1[:,0:5]
  #print(min_cluster.shape)

  # Create the black pixel 'canvas' for the segmentation mask
  min_color_mask = np.zeros((height, width, 3))
  maj_color_mask = np.zeros((height, width, 3))

  # Map the pixels in the smaller cluster onto the 'canvas'
  for i, coords in enumerate(min_cluster[:, 3:]):
    coords[0], coords[1] = coords[1], coords[0]
    min_color_mask[tuple(coords)] = [255,255,255]

  for i, coords in enumerate(maj_cluster[:, 3:]):
    coords[0], coords[1] = coords[1], coords[0]
    maj_color_mask[tuple(coords)] = [255,255,255]

  return min_color_mask, maj_color_mask


In [ ]:
def fill_mask(min_color_mask, thickness):
  gray = cv2.cvtColor(min_color_mask.astype(np.uint8), cv2.COLOR_BGR2GRAY)
  ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
  contours, hierarchy = cv2.findContours(binary, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
  min_color_mask = cv2.drawContours(min_color_mask, contours, -1, (255, 255, 255), thickness=thickness)
  return cv2.fillPoly(min_color_mask, pts=contours, color=(255, 255, 255))

In [ ]:
image_path = '/content/drive/MyDrive/Touch Up The Roots/Images'
mediapipe_path = '/content/drive/MyDrive/Touch Up The Roots/Hair Segmentation/MediaPipe/Output'
selfie_multiclass_path = '/content/drive/MyDrive/Touch Up The Roots/Hair Segmentation/SelfieMulticlass/Output'
clipseg_path = '/content/drive/MyDrive/Touch Up The Roots/Hair Segmentation/CLIPSeg/Output'

images = os.listdir(image_path)
mediapipe_masks = os.listdir(mediapipe_path)
selfie_multiclass_masks = os.listdir(selfie_multiclass_path)
clipseg_masks = os.listdir(clipseg_path)

print(len(images), len(mediapipe_masks), len(selfie_multiclass_masks), len(clipseg_masks))

66 66 66 32


In [ ]:
# Sort image and mask folders so that images and masks are matched up
image_tuples = []
mediapipe_tuples = []
selfie_tuples = []
clipseg_tuples = []
for i in range(len(images)):
  image_tuples.append((int(extract_digits(images[i])), images[i]))
  mediapipe_tuples.append((int(extract_digits(mediapipe_masks[i])), mediapipe_masks[i]))
  selfie_tuples.append((int(extract_digits(selfie_multiclass_masks[i])), selfie_multiclass_masks[i]))

image_tuples = sorted(image_tuples, key=lambda x: x[0])
mediapipe_tuples = sorted(mediapipe_tuples, key=lambda x: x[0])
selfie_tuples = sorted(selfie_tuples, key=lambda x: x[0])

images = list(map(lambda x: x[1], image_tuples))
mediapipe_masks = list(map(lambda x: x[1], mediapipe_tuples))
selfie_multiclass_masks = list(map(lambda x: x[1], selfie_tuples))

for i in range(len(clipseg_masks)): #images
  clipseg_tuples.append((int(extract_digits(clipseg_masks[i])), clipseg_masks[i]))
clipseg_tuples = sorted(clipseg_tuples, key=lambda x: x[0])
clipseg_masks = list(map(lambda x: x[1], clipseg_tuples))


In [ ]:
# mediapipe_output_path = '/content/drive/MyDrive/Touch Up The Roots/Minority Color Hair Segmentation/KMeans on RGB/Expanded Mask (Mediapipe)'

# # Generate minority color segmentation masks from MediaPipe hair segmentation masks
# for i in range(len(images)):
#   img_path = os.path.join(image_path, images[i])
#   mask_path = os.path.join(mediapipe_path, mediapipe_masks[i])
#   hair_mask = load_and_preprocess(img_path, mask_path)
#   min_color_mask = generate_mask(hair_mask)
#   min_color_mask = fill_mask(min_color_mask, thickness=5)

#   # Write image to output folder
#   img_num = extract_digits(images[i])
#   cv2.imwrite(f'{mediapipe_output_path}/min_color_mask{img_num}.jpg', min_color_mask)

In [ ]:
# selfie_multiclass_output_path = '/content/drive/MyDrive/Touch Up The Roots/Minority Color Hair Segmentation/KMeans on RGB/Expanded Mask (SelfieMulticlass)'

# # Generate minority color segmentation masks from SelfieMulticlass hair segmentation masks
# for i in range(len(images)):
#   img_path = os.path.join(image_path, images[i])
#   mask_path = os.path.join(selfie_multiclass_path, selfie_multiclass_masks[i])
#   hair_mask = load_and_preprocess(img_path, mask_path)
#   min_color_mask = generate_mask(hair_mask)
#   min_color_mask = fill_mask(min_color_mask, thickness=5)

#   # Write image to output folder
#   img_num = extract_digits(images[i])
#   cv2.imwrite(f'{selfie_multiclass_output_path}/min_color_mask{img_num}.jpg', min_color_mask)

In [ ]:
clipseg_images = []
clipseg_digits = list(map(lambda x: x[0], clipseg_tuples))
for i in images:
  if int(extract_digits(i)) in clipseg_digits:
    clipseg_images.append(i)
len(clipseg_images)

32

In [ ]:
# REDO '/content/drive/MyDrive/Touch Up The Roots/Minority Color Hair Segmentation/KMeans on RGB/Expanded Mask (SelfieMulticlass)'
clipseg_output_path = '/content/drive/MyDrive/Touch Up The Roots/Minority Color Hair Segmentation/KMeans on HSV/Expanded Mask (CLIPSeg)'
# Generate minority color segmentation masks from SelfieMulticlass hair segmentation masks
for i in range(len(clipseg_images)):
  img_path = os.path.join(image_path, clipseg_images[i])
  mask_path = os.path.join(clipseg_path, clipseg_masks[i])
  hair_mask = load_and_preprocess(img_path, mask_path)
  min_color_mask = generate_mask(hair_mask)
  min_color_mask = fill_mask(min_color_mask, thickness=5)

  # Write image to output folder
  img_num = extract_digits(clipseg_images[i])
  cv2.imwrite(f'{clipseg_output_path}/min_color_mask{img_num}.jpg', min_color_mask)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


AttributeError: 'tuple' object has no attribute 'astype'

In [ ]:
# Path of test image
test_image_path = os.path.join(image_path, images[0])

# Outputs colored hair segmentation mask
test_mask = load_and_preprocess(test_image_path, os.path.join(mediapipe_path, mediapipe_masks[0]))

# Generate gray hair segmentation mask
gray = cv2.cvtColor(load_and_preprocess(test_image_path, os.path.join(mediapipe_path, mediapipe_masks[0])), cv2.COLOR_BGR2GRAY)

# Generate min and maj color masks
min, maj = generate_mask(test_mask)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
colored_min = np.zeros_like(test_mask)
colored_min[np.where(min > 0)] = test_mask[np.where(min > 0)]

colored_maj = np.zeros_like(test_mask)
colored_maj[np.where(maj > 0)] = test_mask[np.where(maj > 0)]

In [ ]:
def loss(image, min_color_mask, maj_color_mask):
  """
  This function computes the Wasserstein distance between the probability distributions of (grayscale) pixel intentsities between the minority and majority color segmentation masks.

  In theory, having the touched up region's pixel color distribution appear as similar as possible to that of the majority color region is desirable because
  the touched up region will blend in better with the remainder of the hair. This improves the color while preserving more "naturalness".  Natural hair texture is often lost because it
  appears that the hair texture in an image is numerically represented by particular patterns of minor variations in RGB values (the RGB values of which are approximated by grayscale pixel intensity)
  and that these patters are vulnerable to being "smoothed" out of existence by the Stable Diffusion model.  Attempting to optimize this loss function is attempting to minimize the amount of
  "smoothness" afflicting the touched up region while pushing the mean (and median) pixel color in the direction of the majority color region.


  Parameters:
  image (numpy array): The original image.
  min_color_mask (numpy array): The minority color segmentation mask.
  maj_color_mask (numpy array): The majority color segmentation mask.

  Returns:
  loss (float): The Wasserstein distance between the probability distributions of (grayscale) pixel intentsities between the minority and majority color segmentation masks.
  """

  # Create black background image equal in size to the original image
  colored_min_region = np.zeros_like(image)
  colored_maj_region = np.zeros_like(image)

  # Extract the indices of the pixel regions of each mask
  min_mask_regions = np.where(min_color_mask > 0)
  maj_mask_regions = np.where(maj_color_mask > 0)

  # Map the colored pixels from the minority and majority color segmentation masks onto the black background image
  colored_min_region[min_mask_regions] = image[min_mask_regions]
  colored_maj_region[maj_mask_regions] = image[maj_mask_regions]

  """
  The two lines below convert RGB images to grayscale images.
  NOTE: The formula to do, based on relative perception of color brightness, is the following: 0.299 ∙ Red + 0.587 ∙ Green + 0.114 ∙ Blue
  The corresponding grayscale pixel value is a weighted sum of the RGB pixel values and represents the pixel "intensity".
  """
  gray_min_region = cv2.cvtColor(colored_min_region, cv2.COLOR_BGR2GRAY)
  gray_maj_region = cv2.cvtColor(colored_maj_region, cv2.COLOR_BGR2GRAY)

  # Flatten each greyscale image to a vector
  gray_min_vector = gray_min_region.flatten()
  gray_maj_vector = gray_maj_region.flatten()

  # Extract nonzero pixel values (ie discard the black background pixels)
  gray_min_pixels = gray_min_vector[gray_min_vector > 0]
  gray_maj_pixels = gray_maj_vector[gray_maj_vector > 0]

  # Compute histogram of pixel intensities for the minority color segmentation mask
  min_mask_distribution = np.bincount(gray_min_pixels) / gray_min_pixels.size

  # Compute histogram of pixel intensities for the majority color segmentation mask
  maj_mask_distribution = np.bincount(gray_maj_pixels) / gray_maj_pixels.size

  # Ensure that the probability distribtion sums to 1
  assert(min_mask_distribution.sum() == 1)
  assert(maj_mask_distribution.sum() == 1)

  # Compute Wasserstein distance between the two histograms of pixel intensities
  distribution_distance = scipy.stats.wasserstein_distance(min_mask_distribution, maj_mask_distribution, u_weights=None, v_weights=None)
  loss = distribution_distance

  return loss

In [ ]:
loss(test_mask, min, maj)

# visualize histogram of empirical counts?

0.003999500062492188

In [ ]:
min

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       ...,

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        ...,
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])